In [118]:
import dlib
import cv2
face_detector = dlib.get_frontal_face_detector()
img = cv2.imread('./Dataset/face/hand2.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
faces = face_detector(img)
size = 80

# print("{} faces are detected.".format(len(faces)))
for f in faces:
    print("left, top, right, bottom : ", f.left(), f.top(), f.right(), f.bottom())
    # cv2.rectangle(img, (f.left(), f.top()), (f.right(), f.bottom()), (0,0,255),2)
    cv2.rectangle(img, ((f.left()-size), (f.top()-size)), ((f.right()+size),(f.bottom()+size)), (0,0,255),2)

# win = dlib.image_window()
# win.set_image(img)
# win.add_overlay(faces)

crop = img[(f.top()-size):(f.bottom()+size), (f.left()-size):(f.right()+size)]
cv2.imwrite("output.jpg", crop, [cv2.IMWRITE_JPEG_QUALITY, 80])

left, top, right, bottom :  221 109 293 181


True

In [1]:
import dlib
import cv2
import glob
from PIL import Image

face_detector = dlib.get_frontal_face_detector()
crop_count = 0
data_path = './Dataset/Train/'

#데이터 로드
face_set = glob.glob('./Dataset/face/*.jpg')

# img = cv2.imread('./Dataset/face/hand1.jpg')
for i in range(len(face_set)):
    #사이즈 비율 정하기
    img = Image.open(face_set[i])
    width, height = img.size

    #이미지 데이터 1개씩 로드
    img = cv2.imread(face_set[i])
    img_area = width * height

    #얼굴 디텍트
    faces = face_detector(img,1)
    
    #디텍트 된 얼굴 수에 따른 반응 설정
    if len(faces) >= 1:
        print("얼굴 검출 성공 =>", face_set[i])
        for i in range(len(faces)):
            # print("left, top, right, bottom : ", f.left(), f.top(), f.right(), f.bottom())

            # 얼굴 크롭할 비율 정하기
            # width = abs(faces[i].left() - faces[i].right())
            # height = abs(faces[i].top() - faces[i].bottom())
            # area = width * height
            # # print('img_area',img_area)
            # # print('area',area)
            # # size = int((img_area/area)*3)
            size = 0

            crop = img[(faces[i].top()-size):(faces[i].bottom()+size), (faces[i].left()-size):(faces[i].right()+size)]
            file_name = str(crop_count)+".jpg"
            data_path_a = data_path + file_name
            #이미지 크기 조정
            crop = cv2.resize(crop, (440, 400), interpolation = cv2.INTER_CUBIC)
            cv2.imwrite(data_path_a, crop)
            crop_count += 1

    elif len(faces) == 0:
            print('얼굴 검출 실패 =>',face_set[i])
            pass
        
print(len(face_set),'장의 사진 중',crop_count,'개의 얼굴 검출, Crop 완료')



얼굴 검출 성공 => ./Dataset/face\apink1.jpg
얼굴 검출 성공 => ./Dataset/face\apink2.jpg
얼굴 검출 성공 => ./Dataset/face\golf.jpg
얼굴 검출 성공 => ./Dataset/face\hand1.jpg
얼굴 검출 성공 => ./Dataset/face\hand2.jpg
얼굴 검출 성공 => ./Dataset/face\kimgunwoo.jpg
얼굴 검출 실패 => ./Dataset/face\p1.jpg
얼굴 검출 성공 => ./Dataset/face\p2.jpg
얼굴 검출 성공 => ./Dataset/face\p3.jpg
9 장의 사진 중 14 개의 얼굴 검출, Crop 완료


In [106]:
a = cv2.imread('./Dataset/face/apink1.jpg')

In [78]:
img = cv2.imread(face_set[8])
faces = face_detector(img)
len(faces)

2

In [ ]:
import dlib
import cv2
import glob
import os
import sys

face_detector = dlib.get_frontal_face_detector()
crop_count = 0
data_path = './Dataset/Train/'
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')
faces_folder_path = './Dataset/face/'

#데이터 경로 로드
face_set = glob.glob(os.path.join(faces_folder_path, "*.jpg")

#이미지 데이터 1개씩 로드
for i in range(len(face_set)):
    img = dlib.load_rgb_image(face_set[i])
    
    #얼굴 디텍트
    faces = face_detector(img,1)
    print("Number of faces detected: {}".format(len(faces)))

for k, d in enumerate(faces):
    # k 얼굴 인덱스
    # d 얼굴 좌표
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))

    # 인식된 좌표에서 랜드마크 추출 
    shape = predictor(img, d)

    # shape는 "full_object_detection" 클래스이다. 
    # part(0)부터 part(67)까지 총 68개의 X,Y 좌표를 가지고 있다. 
    print("Part 0: {}, Part 1: {} ...".format(shape.part(0), shape.part(1)))


    #디텍트 된 얼굴 수에 따른 반응 설정
    if len(faces) >= 1:
        print("얼굴 검출 성공 =>", face_set[i])
        for i in range(len(faces)):
            # print("left, top, right, bottom : ", f.left(), f.top(),
            #  f.right(), f.bottom())
            size = 0

            crop = img[(faces[i].top()-size):(faces[i].bottom()+size),
             (faces[i].left()-size):(faces[i].right()+size)]
            file_name = str(crop_count)+".jpg"
            data_path_a = data_path + file_name
            #이미지 크기 조정
            crop = cv2.resize(crop, (440, 400), interpolation = cv2.INTER_CUBIC)
            cv2.imwrite(data_path_a, crop)
            crop_count += 1

    elif len(faces) == 0:
            print('얼굴 검출 실패 =>',face_set[i])
            pass
        
print(len(face_set),'장의 사진 중',crop_count,'개의 얼굴 검출, Crop 완료')

